MRIB algorithm

In [ ]:
'''
The leg is structured as follows:
(trip_id, departure_node, departure_time, arrival_node, arrival_time, route_id, service_id)

trip_id = leg.iloc[0]
departure_node = leg[1]
departure_time = leg[2]
arrival_node = leg[3]
arrival_time = leg[4]
route_id = leg[5]
service_id = leg[6]
'''

Import Functions

In [5]:
from datetime import datetime, timedelta
import pandas as pd
import math
import scipy.stats as stats
from data_preparation import prepare_data,import_data
pd.set_option('display.max_colwidth', None)

Data Import

In [6]:
agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df,calendar_dates_df = import_data()
legs_df = prepare_data(stops_df,trips_df,stop_times_df)

Reliability Functions

In [ ]:
# Function to calculate the probability of a successful transfer between two subsequent legs
def calculate_transfer_probability(prev_leg: tuple, next_leg: tuple) -> float:
    # Check if the previous leg and the next leg have the same route ID
    if prev_leg[5] == next_leg[5]:
        # If they do, the transfer is guaranteed (probability = 1)
        return 1
    else:

        prev_arrival_time = datetime.strptime(prev_leg[4], "%H:%M:%S")
        next_departure_time = datetime.strptime(next_leg[2], "%H:%M:%S")
        
        # Calculate the transfer time in minutes
        transfer_time = (next_departure_time - prev_arrival_time).total_seconds() / 60
        # Calculate the probability using the CDF of a Gamma distribution 
        return min(stats.gamma.cdf(transfer_time, a=2, scale=4),0.95)


# Function to calculate cumulative probabilities for a given itinerary
def calculate_cumulative_probability(itinerary:list) -> list[float]:
    # Initialize a list to store cumulative probabilities, starting with 1 (100% probability for the first leg)
    cumulative_probabilities = [1] 
    
    # Iterate through each pair of consecutive legs in the itinerary
    for i in range(len(itinerary) - 1):
        # Get the current leg (prev_leg) and the next leg (next_leg)
        prev_leg = itinerary[i]
        next_leg = itinerary[i + 1]

        # Calculate the probability of a successful transfer between the two legs
        transfer_prob = calculate_transfer_probability(prev_leg, next_leg)

        # Append the transfer probability to the cumulative probabilities list
        cumulative_probabilities.append(transfer_prob)

    return cumulative_probabilities

# Function to calculate the probability of arriving within the time budget given successful transfers
# Redundant due to network filtering
def calculate_arrival_probability(itinerary:list, start_time:str, time_budget:timedelta) -> int:
    # Check if all transfers in the itinerary can potentially be successful using cumulative probability
    if math.prod(calculate_cumulative_probability(itinerary)) > 0:  
        # If all transfers are successful, proceed to calculate the arrival time of the final leg of the itinerary
        destination_leg = itinerary[-1] 
        destination_arrival_time = destination_leg[4]  
        
        # Convert start time and destination arrival time to datetime objects for comparison
        start_time = datetime.strptime(start_time, "%H:%M:%S")
        destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

        # Calculate total travel time
        total_travel_time = destination_arrival_time - start_time

        # Check if the total travel time is within the specified time budget
        if total_travel_time <= time_budget:
            return 1  # Return 1 if arrival is within the time budget
        else:
            return 0  # Return 0 if arrival is beyond the time budget
    else:
        # Return 0 if any transfer in the itinerary is not possible
        return 0


# Function to calculate the reliability of a primary itinerary
def primary_itinerary_reliability(itinerary:list, start_time:str, time_budget:timedelta) -> float:
    '''
    Compute the overall reliability as the product of 
     - Arrival probability
     - Pruduct of cumulative probabilities of making a transfer
     '''
    
    # Calculate the cumulative probabilities for all transfers in the itinerary
    cumulative_probabilities = calculate_cumulative_probability(itinerary)
    
    # Compute the product of cumulative probabilities to determine the overall transfer success probability
    product_of_probabilities = math.prod(cumulative_probabilities)
    
    # Calculate the probability of arriving within the time budget
    arrival_probability = calculate_arrival_probability(itinerary, start_time, time_budget)
    
    reliability = arrival_probability * product_of_probabilities

    return reliability


# Function to calculate the reliability of a backup itinerary
# backup = (transfer leg of the primary itinerary, [sequence of legs of backup starting from the next after transfer], reliability)
def backup_itinerary_reliability(itinerary:list, backup: tuple, start_time:str, time_budget:timedelta) -> float:
    '''
     Calculate the backup reliability as the product of:
     - Arrival probability for the backup itinerary
     - Product of cumulative probabilities for the backup itinerary
     - Probability of a missing a transfer (1 - initial_transfer_prob)
     - Reliability of the primary itinerary up to the transfer point
    '''
    # Extract the sequence of legs for the backup itinerary
    backup_itinerary = backup[1][:]

    # Calculate the arrival and cumulative probabilities of the backup route
    arrival_probability = calculate_arrival_probability(backup_itinerary, start_time, time_budget)
    cumulative_probabilities = calculate_cumulative_probability(backup_itinerary)
    product_of_probabilities = math.prod(cumulative_probabilities)

    # Extract the transfer leg from the primary itinerary
    transfer_leg = backup[0]

    # Default initial transfer probability in case no missed transfer is identified
    initial_transfer_prob = 1

    # Identify the leg in the primary itinerary where the backup starts
    for idx, leg in enumerate(itinerary[:-1]):  
        if leg[3] == transfer_leg[3]:  
            prev_leg = itinerary[idx]  # The transfer leg 
            missed_leg = itinerary[idx + 1]  # The possibly missed leg after the transfer
            # Calculate the transfer probability for the transfer in the primary itinerary
            initial_transfer_prob = calculate_transfer_probability(prev_leg, missed_leg)
            # Calculate the reliability of the primary itinerary up to the point of the transfer
            primary_itinerary_rel_before_transfer = primary_itinerary_reliability(itinerary[:idx+1], start_time, time_budget)
            break  
    
    backup_reliability = (arrival_probability * product_of_probabilities * (1 - initial_transfer_prob) * primary_itinerary_rel_before_transfer)

    return backup_reliability


# Function to calculate the reliability of a complete itinerary, including primary and backup itineraries
def itinerary_reliability(itinerary: list, Backups: list[tuple], start_time: str, time_budget: timedelta) -> float:
    '''
     Calculate the total reliability as 
     - Primary itinerary reliability
     - Backup itineraries reliability
    '''
    # Calculate the reliability of the primary itinerary
    primary_reliability = primary_itinerary_reliability(itinerary, start_time, time_budget)

    # If the primary itinerary has non-zero reliability
    if primary_reliability > 0:
        added_reliability = 0  # Initialize added reliability from backups to 0

        # Iterate through all backup itineraries
        for backup in Backups:
            # Calculate the reliability of the current backup itinerary
            backup_reliability = backup_itinerary_reliability(itinerary, backup, start_time, time_budget)
            # Accumulate the reliability from all backups
            added_reliability += backup_reliability

        complete_reliability = primary_reliability + added_reliability

        # Return the calculated complete reliability
        return complete_reliability
    else:
        # If the primary itinerary reliability is 0, return 0
        return 0.0



Network Filtering

In [8]:
# Function to get the available service IDs for a given start date
def get_available_service_ids(start_date:str) -> list: 
    start_date_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    start_date_str = start_date_datetime.strftime("%Y%m%d")

    # Determine the weekday name for the given date
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][start_date_datetime.weekday()]

    # Initialize a list to store available service IDs
    available_service_ids = []

    # Check for regular service in the calendar dataframe
    for _, service in calendar_df.iterrows():
        service_id = service["service_id"]
        
        # Check if the given date is within the service's active date range
        if int(service["start_date"]) <= int(start_date_str) <= int(service["end_date"]):
            # Check if the service operates on the determined weekday
            if service[weekday] == 1:
                # Add the service ID to the list if not already present
                if service_id not in available_service_ids:
                    available_service_ids.append(service_id)

    # Check for exceptions in the calendar_dates dataframe
    # Filter the calendar_dates dataframe to only include rows matching the given date
    exceptions = calendar_dates_df[calendar_dates_df["date"] == int(start_date_str)]
    
    # Process each exception to adjust the list of available services
    for _, exception in exceptions.iterrows():
        service_id = exception["service_id"]
        
        if exception["exception_type"] == 2:  # Service is added as an exception
            # Add the service ID to the list if not already present
            if service_id not in available_service_ids:
                available_service_ids.append(service_id)
                
        elif exception["exception_type"] == 1:  # Service is removed as an exception
            # Remove the service ID from the list if it exists
            if service_id in available_service_ids:
                available_service_ids.remove(service_id)

    return available_service_ids


# Function to filter the network of legs based on time frame and available services
def filter_network(start_time:str, start_date:str, time_budget:timedelta) -> list:
    # Retrieve the list of available service IDs for the given date
    available_services = get_available_service_ids(start_date)

    # Initialize an empty list to store the filtered legs
    filtered_network = []

    # Calculate the end time based on the given time budget
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    end_time = start_time + time_budget

    # Iterate through all rows in the `legs_df` dataframe
    for row in legs_df:
        # Extract the departure and arrival times for the current leg
        leg_departure_time = datetime.strptime(row[2], "%H:%M:%S")
        leg_arrival_time = datetime.strptime(row[4], "%H:%M:%S")

        # Check if the leg's departure and arrival times are within the time window
        if (
            leg_departure_time >= start_time  # Departure is after or at the start time
            and leg_departure_time <= end_time  # Departure is before or at the end time
            and leg_arrival_time <= end_time  # Arrival is before or at the end time
        ):
            # Extract the service ID for the current leg
            service_id = row[6]

            #  Check if the service ID is in the list of available services
            if service_id in available_services:
                # Add the leg to the filtered network
                filtered_network.append(row)

    return filtered_network


Helping functions

In [17]:
# Function to search for adjacent legs based on arrival node and time
def search_adjecent_legs(arrival_node:str, arrival_time:str, filtered_legs:list) -> list:
    # Convert the arrival_time to datetime format for comparison
    arrival_time = datetime.strptime(arrival_time, "%H:%M:%S")
    
    # Initialize an empty list to store adjacent legs
    adjecent_legs = []

    # Iterate through each filtered leg
    for i in range(0, len(filtered_legs)):
        leg = filtered_legs[i]

        # Convert the departure time of the current leg to datetime format
        leg_departure_time = datetime.strptime(leg[2], "%H:%M:%S")

        # Check if the leg starts from the given node and departs after the arrival time
        if leg[1] == arrival_node and leg_departure_time >= arrival_time:
            # Add the leg to the list of adjacent legs
            adjecent_legs.append(leg)

    return adjecent_legs

# Function to check if the last two legs in the itinerary involve a transfer
def is_transfer(itinerary:list) -> bool:
    # Get the second last leg in the itinerary (previous leg)
    prev_leg = itinerary[-2]
    
    # Get the last leg in the itinerary (next leg)
    next_leg = itinerary[-1]
    print(prev_leg[5],next_leg[5])
    # Check if both legs have the same tripId (no transfer if they are the same)
    if prev_leg[5] == next_leg[5]:
        return False
    else:
        return True


# Function to calculate the total travel time for the itinerary
def travel_time(itinerary:list, start_time:str):
    # Get the final leg of the itinerary (destination leg)
    destination_leg = itinerary[-1]
    
    # Extract the arrival time of the destination leg
    destination_arrival_time = destination_leg[4]
        
    # Convert the start time and destination arrival time to datetime objects
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

    # Calculate the total travel time by subtracting start time from destination arrival time
    total_travel_time = destination_arrival_time - start_time
    
    return total_travel_time

# Function to find the index of the itinerary with the shortest duration
def find_min_index(LIST: list) -> int:
    
    # Initialize the minimum value with the duration of the first itinerary
    min_index = 0
    min_value = LIST[0][2]  # Setting the min valie to the duration of the first itinerary duration
    
    # Iterate over the remaining itineraries to find the one with the shortest duration
    for i in range(1, len(LIST)):
        current_value = LIST[i][2]  # Get the duration of the current itinerary
        
        # If the current duration is less than the minimum value, update the minimum value and index
        if current_value < min_value:
            min_value = current_value
            min_index = i

    return min_index

# Function to check and update the most reliable itinerary path (MRIB) if a better one is found
def check_and_update_mrib(shortest_path:list, MRIB_reliability:float, MRIB:list, start_time:str, time_budget:timedelta):
    
    # Extract the backup paths from the shortest path
    Backups = shortest_path[4][:]

    # Calculate the reliability of the itinerary 
    rel = itinerary_reliability(shortest_path[0], Backups, start_time, time_budget)

    # Check if the current path has a higher reliability than the existing MRIB
    if rel > MRIB_reliability:
        # Update MRIB and its reliability if the current path is more reliable
        MRIB_reliability = rel
        MRIB = shortest_path

    return MRIB_reliability, MRIB


Output transformation

In [18]:
def transform_route_info(MRIB,MRIB_reliability):
    primary_itinerary = MRIB[0]  # Primary itinerary (first item in best_result_fast)
    reliability = MRIB_reliability  # Reliability score
    duration = MRIB[2]  # Duration of the trip
    arrival_time = MRIB[3]  # Arrival time
    backups = MRIB[4]  # Backup routes
    
    # Initialize the grouped_routes list
    grouped_routes = []
    last_route = None  # To track the previous route for grouping

    # Step 1: Process the primary itinerary
    for i in range(len(primary_itinerary)):
        current_stop = primary_itinerary[i][1]
        route_id = primary_itinerary[i][5]
        departure_time = primary_itinerary[i][2]
        arrival_time = primary_itinerary[i][4]
        next_stop = primary_itinerary[i][3]

        # Grouping the routes based on the route ID
        if route_id == last_route:
            grouped_routes[-1]["stops"].append((next_stop, arrival_time))
        else:
            grouped_routes.append({
                "route_id": route_id,
                "start_stop": current_stop,
                "departure_time": departure_time,
                "stops": [(next_stop, arrival_time)]
            })
        last_route = route_id

    # Step 2: Print the grouped routes for the primary itinerary
    for segment in grouped_routes:
        start = segment["start_stop"]
        dep_time = segment["departure_time"]
        route = segment["route_id"]
        stops = " → ".join([f"{stop} (Arrival: {arr})" for stop, arr in segment["stops"]])
        print(f"  🚆 {start} (Departure: {dep_time}) → {stops} with Line {route}")

    # Step 3: Print additional details
    print(f"\n🎯 End station: {MRIB[-2]} (Arrival: {arrival_time})")
    print(f"🔹 Total route reliability: {reliability:.2f}\n")
    
    # Step 4: Process and print the backup routes
    if backups:
        print("🔄 Backups:")
        for backup in backups:
            primary_path = backup[0]
            backup_path = backup[1]
            backup_reliability = backup[2]
            grouped_backup_routes = []
            last_backup_route = None

            for i in range(len(backup_path)):
                current_stop = backup_path[i][1]
                route_id = backup_path[i][5]
                departure_time = backup_path[i][2]
                arrival_time = backup_path[i][4]
                next_stop = backup_path[i][3]

                # Group backup routes by route ID
                if route_id == last_backup_route:
                    grouped_backup_routes[-1]["stops"].append((next_stop, arrival_time))
                else:
                    grouped_backup_routes.append({
                        "route_id": route_id,
                        "start_stop": current_stop,
                        "departure_time": departure_time,
                        "stops": [(next_stop, arrival_time)]
                    })
                last_backup_route = route_id

            # Print backup route segments
            for segment in grouped_backup_routes:
                start = segment["start_stop"]
                dep_time = segment["departure_time"]
                route = segment["route_id"]
                stops = " → ".join([f"{stop} (Arrival: {arr})" for stop, arr in segment["stops"]])
                print(f"  🚆 {start} (Departure: {dep_time}) → {stops} with Line {route}")

            print(f"🔹 Total reliability of backup routes: {backup_reliability:.2f}\n")
            


Backup Search 

In [30]:
def backup_search(shortest_path:list, shortest_next_itinerary:list, destination_node:str, start_time:str, time_budget:timedelta, filtered_legs:list):
    """
    Search for backup itineraries, calculate their reliability, and determine the most reliable backup path (MRB).

    Input:
        shortest_path (tuple): The itinerary including the transfer leg.
        shortest_next_itinerary (list): The itinerary including the transfer leg and next leg(missed connection). 
        destination_node (str): The destination node of the itinerary.
        start_time (str): The start time.
        time_budget (timedelta): The maximum allowed time for travel.
        filtered_legs (list): A list of filtered legs representing available legs between nodes.

    Output:
        - MRB (tuple): The most reliable backup itinerary.
        - MRB_reliability (float): The reliability of the most reliable backup itinerary.
    """
    
    '''Initial setup'''
    MRB_reliability = 0  # Most reliable backup reliability (initially 0)
    MRB = None  # Placeholder for the most reliable backup itinerary
    LIST_Backups = []  # List to store potential backup itineraries
    
    # Extract the transfer leg from the shortest path and calculate the missed departure time
    transfer_leg = shortest_path[0][-1]
    transfer_point = transfer_leg[3]
    primary_itinerary = shortest_next_itinerary[0]
    missed_leg_dep_time = datetime.strptime(primary_itinerary[-1][2], "%H:%M:%S")  # Departure time of missed leg

    # Track the stops passed in the backup path
    passed_stops_b = []
    for leg in primary_itinerary[:-1]: #Except the missed leg
        passed_stops_b.append(leg[3])  # Append each stop in the primary itinerary
    passed_stops_b.append(primary_itinerary[0][1])  # Add the start node of the primary itinerary
    
    ''' Search initial adjacent legs'''
    # Find adjacent legs that can be considered for backup itineraries
    adjecent_legs = search_adjecent_legs(transfer_point, primary_itinerary[-1][2], filtered_legs) #arrival node, arrival time to the transfer point
    adjecent_legs = [leg for leg in adjecent_legs if leg[3] not in passed_stops_b]  # Filter out already passed stops
    adjecent_legs = [leg for leg in adjecent_legs if datetime.strptime(leg[2], "%H:%M:%S") > missed_leg_dep_time]  # Only include legs after the missed departure time
    
    # Create backup itineraries based on the available adjacent legs
    for leg in adjecent_legs: 
        backup = (transfer_leg, [leg])  
        b_reliability = backup_itinerary_reliability(primary_itinerary, backup, start_time, time_budget)
        backup_full = (transfer_leg, [leg], b_reliability)  
        b_duration = travel_time([leg], start_time)  #
        if b_reliability > 0 and timedelta(seconds=0) < b_duration <= time_budget:  # Check if backup is valid
            LIST_Backups.append(backup_full)  

    '''Main Loop'''
    while len(LIST_Backups) > 0:
        min_index_b = 0
        min_value_b = LIST_Backups[0][1][-1][4]  # Get the arrival time of the last leg of the first backup
        for i in range(1, len(LIST_Backups)):
            current_value_b = LIST_Backups[i][1][-1][4]  # Compare arrival time of the current backup
            if current_value_b < min_value_b:
                min_value_b = current_value_b
                min_index_b = i
        
        # Pop the backup with the earliest arrival time
        shortest_backup = LIST_Backups.pop(min_index_b)

        # Add the nodes of the current backup to the passed stops
        for leg in shortest_backup[1]:
            destination_stop = leg[3]
            passed_stops_b.append(destination_stop)

        b_tail = shortest_backup[1][-1]  # Get the last leg in the backup

        # Check if the last leg of the backup reaches the destination node
        if b_tail[3] == destination_node:
            rel = backup_itinerary_reliability(shortest_next_itinerary[0], shortest_backup, start_time, time_budget)
            if rel > MRB_reliability:  # Update MRB if a more reliable backup is found
                MRB_reliability = rel
                MRB = shortest_backup
        else:
            # Find adjacent legs from the last leg of the current backup
            next_legs_b = search_adjecent_legs(b_tail[3], b_tail[4], filtered_legs)

            next_legs_b = [leg for leg in next_legs_b if leg[3] not in passed_stops_b]  # Filter out already passed stops
            # Ensure that the deparute time is bugger that arrival, it transfer occurs
            next_legs_b = [
            leg for leg in next_legs_b 
            if leg[0] == b_tail[0] or datetime.strptime(leg[2], "%H:%M:%S") > (datetime.strptime(b_tail[4], "%H:%M:%S") + timedelta(minutes=2))
        ]
            # Add new backup legs to the list of backups
            for leg in next_legs_b:
                backup_legs = shortest_backup[1][:]  # Copy the current backup legs
                backup_legs.append(leg)  # 
                backup = (transfer_leg, backup_legs)
                b_reliability = backup_itinerary_reliability(shortest_next_itinerary[0], backup, start_time, time_budget)
                backup_full = (transfer_leg, backup_legs, b_reliability)
                b_duration = travel_time(backup_legs, start_time)  # Calculate the travel time for the new backup
                if b_reliability > MRB_reliability and timedelta(seconds=0) < b_duration <= time_budget:
                    LIST_Backups.append(backup_full)  # Add valid backup to the list

    # End of the backup search loop
  
    return MRB, MRB_reliability  # Return the most reliable backup and its reliability


Main Algorithm

In [ ]:
def find_path(origin_node: str,destination_node: str, start_datetime : str, time_budget: timedelta):
    '''
    Finds the most reliable itinerary within a given time budget between two nodes.
    
    Parameters:
    - origin_node (str): The starting node of the journey.
    - destination_node (str): The target node of the journey.
    - start_datetime (str): The starting date and time in 'YYYY-MM-DD HH:MM:SS' format.
    - time_budget (timedelta): The maximum allowed travel duration.

    Returns:
    - MRIB_reliability (float): The reliability of the most reliable itinerary.
    - MRIB (list): The most reliable itinerary details.
    '''
     
    '''Initial setup'''
    start_date, start_time = start_datetime.split()

    filtered_legs = filter_network(start_time,start_date,time_budget)
    MRIB_reliability = 0.0
    MRIB = None
    LISTofTRIPS = []
    n = 0 #counting variable


    ''' Search initial adjacent legs'''
    filtered_adj_legs = search_adjecent_legs(origin_node,start_time,filtered_legs)
    for leg in filtered_adj_legs:
        itinerary  = [leg] # Start with a single leg itinerary
        reliability = 1
        duration = travel_time(itinerary,start_time)
        expected_arrival_time = leg[4] #arrival time
        Backups = [] # Initialize an empty backup list

        # Append trip to LISTofTRIPS if valid
        if reliability > 0 and timedelta(seconds=0) < duration <= time_budget: #prevents negative duration 
            LISTofTRIPS.append([itinerary,reliability,duration,expected_arrival_time,Backups])
        
    '''Main Loop'''
    while LISTofTRIPS:
        
        MRB = None
        MRB_departure_time = None   # To store the MRB's departure time
        first_iteration = True 
        n +=1 #Track loop iterations
        
        print("New iteration", n) # Keep track of iteration 
    
        #Add origin node to the passed stops
        passed_stops_n = [origin_node]

        # Find the shortest itinerary based on the current list
        min_index = find_min_index(LISTofTRIPS)
        shortest_path = LISTofTRIPS.pop(min_index)

        tail = shortest_path[0][-1] # last leg of trip
        
        # Check if destination is reached
        if tail[3] == destination_node:
            MRIB_reliability, MRIB = check_and_update_mrib(shortest_path, MRIB_reliability, MRIB, start_time, time_budget)
            continue
        
        # Track stops already in the current path
        for leg in shortest_path[0]: 
            destination_stop = leg[3] #arrival node
            passed_stops_n.append(destination_stop)
        
        # Search for adjacent legs from the tail's destination
        next_legs = search_adjecent_legs(tail[3],tail[4],filtered_legs) #tail node, tail arrival time

        # Filter out legs leading to already visited stops
        next_legs = [leg for leg in next_legs if leg[3] not in passed_stops_n]

        # If transfer occurs, ensure that departure is bigger than arrival, except for if stay on the same route
        next_legs = [
            leg for leg in next_legs 
            if leg[0] == tail[0] or datetime.strptime(leg[2], "%H:%M:%S") > (datetime.strptime(tail[4], "%H:%M:%S"))
        ]
        
        # Adding new legs to current path
        for leg in next_legs:
            itinerary = shortest_path[0] + [leg] #Combine previous legs and adjecent
            Backups = shortest_path[4][:] #Transfer the backups
            reliability = itinerary_reliability(itinerary,Backups,start_time,time_budget)
            duration = travel_time(itinerary,start_time)
            expected_arrival_time = leg[4] #arrival time
            trip = [itinerary,reliability,duration,expected_arrival_time,Backups]

            # Check reliability and duration constraints
            if reliability > MRIB_reliability and timedelta(seconds=0) < duration <= time_budget: #
                if is_transfer(trip[0]) is False: #Direct connection
                    LISTofTRIPS.append(trip)
                    
                else: #Transfer point

                    '''Backup loop'''
                     # Perform backup search only when required
                    if first_iteration or (
                        MRB_departure_time and 
                        MRB_departure_time <= trip[0][-1][2] #departure time of last leg of path (the "missed transfer")
                    ):
                        # Ensure valid search index for backup
                        if next_legs.index(leg) + 1 < len(next_legs):
                            MRB, MRB_reliability = backup_search(shortest_path,trip,destination_node,start_time,time_budget,filtered_legs)
                            if MRB:
                                MRB_departure_time = MRB[1][0][2]  # Update MRB's departure time
                        else:
                            MRB = None
                    if MRB : #if the backup for this transfer exists
                        trip[4] = trip[4][:] #copy all previous backups of the path
                        trip[4].append(MRB) # add new backup

                    ''' Check if itinerary is good enought to add it'''
                    next_reliability = itinerary_reliability(trip[0],trip[4],start_time,time_budget)
                    if next_reliability >= MRIB_reliability: # Only append if reliability improves
                        LISTofTRIPS.append(trip)
             
    return MRIB_reliability, MRIB

In [ ]:
#Example of input
origin_node = "Laa an der Thaya"  
destination_node = "Wien Leopoldau" 
start_time = "2024-12-12 09:00:00"
time_budget = timedelta(hours=1, minutes = 30)

MRIB_reliability, MRIB = find_path(origin_node,destination_node,start_time,time_budget)

In [ ]:
# Example of results display
print(transform_route_info(MRIB,MRIB_reliability))